In [1]:
# Imports
import numpy as np
import pandas as pd
import datetime

In [2]:
#data = pd.concat([pd.read_csv('Data/2009.csv'), pd.read_csv('Data/2010.csv'), 
 #                 pd.read_csv('Data/2011.csv'), pd.read_csv('Data/2012.csv'), 
  #                pd.read_csv('Data/2013.csv'), pd.read_csv('Data/2014.csv'), 
   #               pd.read_csv('Data/2015.csv'), pd.read_csv('Data/2016.csv'), 
    #              pd.read_csv('Data/2017.csv'), pd.read_csv('Data/2018.csv')])

# Read 2018 data only
data = pd.read_csv('Data/2010.csv')

In [3]:
# Convert string to DateTime and set as index
data.FL_DATE = pd.to_datetime(data.FL_DATE, infer_datetime_format=True)
data.set_index('FL_DATE', inplace=True)
data.drop(columns='Unnamed: 27', axis=1, inplace=True) # Drop last weird column
data.head()

,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
FL_DATE,,,,,,,,,,,,,,,,,,,,,
2010-01-01,MQ,4438,LGA,RDU,1940,2101.0,81.0,14.0,2115.0,2225.0,...,0.0,110.0,106.0,70.0,431.0,0.0,0.0,0.0,0.0,77.0
2010-01-01,MQ,4440,DCA,JFK,950,949.0,-1.0,13.0,1002.0,1040.0,...,0.0,70.0,60.0,38.0,213.0,NaN,NaN,NaN,NaN,NaN
2010-01-01,MQ,4443,LGA,RDU,900,937.0,37.0,11.0,948.0,1105.0,...,0.0,120.0,91.0,77.0,431.0,NaN,NaN,NaN,NaN,NaN
2010-01-01,MQ,4445,RDU,LGA,1940,1935.0,-5.0,12.0,1947.0,2051.0,...,0.0,100.0,80.0,64.0,431.0,NaN,NaN,NaN,NaN,NaN
2010-01-01,MQ,4447,JFK,DCA,2035,2236.0,121.0,18.0,2254.0,2341.0,...,0.0,85.0,66.0,47.0,213.0,0.0,0.0,0.0,0.0,102.0


In [4]:
# Drop NaN values in the variables we will use in our baseline model
data = data[data['TAXI_OUT'].notna()]
data = data[data['ORIGIN'].notna()]
data = data[data['DEST'].notna()]


In [16]:
# Take first 6 months
df = data.loc['2010-01-01':'2010-06-30']


In [17]:
# Renaming airline codes to company names
# Source: https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States

df['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
},inplace=True)

c:\Users\chris\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [18]:
df = df[df.ORIGIN == 'JFK']

In [19]:
# Take only the delayed flights
df = df[df['CARRIER_DELAY' ] >= 0]

In [20]:
delays = df[['CARRIER_DELAY', 'DEP_TIME']]
delays['DEP_TIME'] = delays['DEP_TIME'].astype(int)

preps = []
for i in range(len(delays['DEP_TIME'])):
    # Zero fill values
    dep_time_val = str(delays['DEP_TIME'].iloc[i]).zfill(4)
    # If flight at 24:00, set that as 00:00
    if dep_time_val == str(2400):
        dep_time_act = datetime.datetime.strptime('0000','%H%M').strftime('%H:%M')
    else:
        dep_time_act = datetime.datetime.strptime(dep_time_val,'%H%M').strftime('%H:%M')
    

    # append
    preps.append(dep_time_act)

# Drop and add corrected times
df.drop(columns=['DEP_TIME'])
df['DEP_TIME'] = preps
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H:%M').dt.time # Convert to datetime

# Convert FL time to column
df = df.reset_index(level=0)

<ipython-input-20-df0267221f87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delays['DEP_TIME'] = delays['DEP_TIME'].astype(int)


In [21]:
print(df['DEP_TIME'])
print(df['FL_DATE'])

0        22:36:00
1        15:17:00
2        11:49:00
3        13:00:00
4        20:00:00
           ...   
10848    08:06:00
10849    21:25:00
10850    09:55:00
10851    15:15:00
10852    21:40:00
Name: DEP_TIME, Length: 10853, dtype: object
0       2010-01-01
1       2010-01-01
2       2010-01-01
3       2010-01-01
4       2010-01-01
           ...    
10848   2010-06-30
10849   2010-06-30
10850   2010-06-30
10851   2010-06-30
10852   2010-06-30
Name: FL_DATE, Length: 10853, dtype: datetime64[ns]


In [22]:
# Compare shape to see effect of preprocessing
print('Shape before:',data.shape,'\n Shape after:',df.shape)

Shape before: (6338168, 26) 
 Shape after: (10853, 27)


In [49]:
df_weather_all = pd.read_csv('Data/Weather.csv')
# Select only relevant features
df_weather = df_weather_all[["NAME","DATE","HLY-TEMP-NORMAL","HLY-WIND-AVGSPD"]]
# Select only JFK airport and from January 1st to July 1st
df_weather= df_weather[(df_weather["NAME"] == "JFK INTERNATIONAL AIRPORT, NY US") & (df_weather["DATE"] < '07-01T00:00:00')]


In [50]:
# Change the format of DATE to datetime
df_weather["DATE"] = pd.to_datetime(df_weather["DATE"],format='%m-%dT%H:%M:%S',infer_datetime_format='%d-%m-%H')
# Change the format to month - day - hour
df_weather["DATE"] = df_weather["DATE"].dt.strftime("%m-%d-%H")
# Change NAME to just JFK
df_weather["NAME"] = 'JFK'
# Reset index
df_weather = df_weather.reset_index().drop('index',axis=1)
df_weather

,NAME,DATE,HLY-TEMP-NORMAL,HLY-WIND-AVGSPD
0,JFK,01-01-01,33.6,11.9
1,JFK,01-01-02,33.4,11.7
2,JFK,01-01-03,33.1,11.4
3,JFK,01-01-04,32.8,11.3
4,JFK,01-01-05,32.7,11.5
...,...,...,...,...
4338,JFK,06-30-19,73.6,11.8
4339,JFK,06-30-20,72.5,10.6
4340,JFK,06-30-21,71.8,9.6
4341,JFK,06-30-22,71.3,9.2


In [88]:
# Select interesting features
df = df[["FL_DATE","OP_CARRIER","DEP_TIME","TAXI_OUT"]]
df

KeyError: "['OP_CARRIER'] not in index"

In [73]:
# Now we must merge df_weather and df
df["TEMP"] = 0
df["WIND"] = 0

for i in range(len(df)):
    # Filter out the month, day and hour 
    hour = df["DEP_TIME"][i].strftime("%H")
    monthday = df["FL_DATE"][i].strftime("%m-%d")
    
    if monthday+'-'+hour == '01-01-00':
        continue
    else:
        # Find temp in df_weather corresponding to month, day and hour
        df.iloc[i,-2] = df_weather[df_weather["DATE"] == monthday+'-'+hour].iloc[0,2]
        # Find wind in df_weather corresponding to month, day and hour
        df.iloc[i,-1] = df_weather[df_weather["DATE"] == monthday+'-'+hour].iloc[0,3]

In [82]:
# The hour of january 1st is not included in weather data. Therefore we drop the flights occurring in that time
df = df[df.TEMP > 0]

In [87]:
df

,FL_DATE,DEST,DEP_TIME,TAXI_OUT,TEMP,WIND
0,2010-01-01,DCA,22:36:00,18.0,34.7,12.4
1,2010-01-01,BOS,15:17:00,18.0,38.7,13.2
2,2010-01-01,RDU,11:49:00,29.0,36.7,13.1
3,2010-01-01,RIC,13:00:00,27.0,38.4,13.2
4,2010-01-01,IND,20:00:00,57.0,35.7,13.0
...,...,...,...,...,...,...
10848,2010-06-30,MSP,08:06:00,31.0,73.1,9.3
10849,2010-06-30,DCA,21:25:00,30.0,71.8,9.6
10850,2010-06-30,BTV,09:55:00,16.0,75.0,9.7
10851,2010-06-30,IAD,15:15:00,34.0,78.6,13.7


In [13]:
# Save as new dataframe
df.to_csv('Data/data_pre.csv')